In [1]:
from fastai import *
from fastai.text import * 

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
df = pd.read_csv('patent-data-extended.csv', header=None, names=['idea'])

In [5]:
df['idea'] = df['idea'].str.lower()

In [6]:
df.head()

,idea
0,a binder and fiber glass product from maillard...
1,a binder and wood board product from maillard ...
2,a capacitive contactless powering system
3,a capacitive touch sensor for identifying a fi...
4,a capacitor construction with oxidation barrie...


In [7]:
valid_pct = 0.05 #validation percent
df = df.iloc[np.random.permutation(len(df))]
cut = int(valid_pct * len(df)) + 1
train_df, valid_df = df[cut:], df[:cut]

In [8]:
len(df)

64649

In [9]:
len(train_df)

61416

In [10]:
len(valid_df)

3233

In [11]:
nan_rows = df[df['idea'].isnull()]

In [12]:
nan_rows

,idea


In [13]:
data_lm = TextLMDataBunch.from_df('data', train_df, valid_df, text_cols='idea')

In [14]:
#data_clas = TextClasDataBunch.from_df('data', train_df, valid_df, text_cols='text', label_cols='author', vocab=data_lm.train_ds.vocab, bs=32)

In [15]:
learn = language_model_learner(data_lm, arch = AWD_LSTM, pretrained=True, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.020906,5.426728,0.233705,06:10


In [16]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.193276,5.072429,0.260848,10:58


In [17]:
wd=1e-7
lr=1e-3
lrs = lr

In [18]:
learn.fit(1,lrs, wd)

epoch,train_loss,valid_loss,accuracy,time
0,4.975592,4.933951,0.267232,11:13


In [19]:
number_of_ideas = 100
ideas_counter = 0
all_ideas = []

for i in range(1):
    idea = learn.predict("xxbos xxfld 1", n_words=5000, temperature=0.8)
    ideas = idea.split("xxbos")
    #ideas = ideas[1:-1]
    
    for idea in ideas:
        idea = idea.replace("xxbos xxfld 1 ","").strip()
        if(idea):
            all_ideas.append(idea)
            ideas_counter = ideas_counter+1
            
    if ideas_counter > number_of_ideas:
        break

In [20]:
all_ideas

['xxfld 1 for a gaming machine',
 'wireless terminal module',
 'foamed tray treadmill with a rear tip',
 'smart track sampling for analog to digital converter',
 'raster with channel processing',
 'storage device with reduced network expression',
 'variable muffler mode and sound',
 'light source device and power supply system',
 'translating conference data in logical databases',
 'tiled display machine',
 'virtual duplex generator',
 'location based input triggering',
 'sequence apparatus for a shower head',
 'associative initialization for network contexts',
 'correlation of experience in architectural frames',
 'housing for a mobile phone communication system',
 'flow fixation system',
 'microphone with link split',
 'user amounts for resistive flash memory data',
 'network accessing network terminals',
 'program advertisement and user interface',
 'uniform gain trough for a rack',
 'circuit for an stimulation circuit',
 'assembly for programmable persistent memory',
 'small block 

In [21]:
learn.save_encoder('ml_ft_enc')

In [22]:
train_df.to_pickle('data/ml_train_df.pkl')

In [23]:
valid_df.to_pickle('data/ml_valid_df.pkl')